In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('parsed.csv')

In [3]:
data.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,status,time,title,tsunami,type,types,tz,updated,url,parsed_place
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...,California
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...,California
2,NaN,4.4,37389194,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.021370,28.0,21.0,",ci37389194,",3.42,ml,...,automatic,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...,California
3,NaN,NaN,37389186,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.026180,NaN,39.0,",ci37389186,",0.44,ml,...,automatic,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...,California
4,NaN,NaN,73096941,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.077990,NaN,192.0,",nc73096941,",2.16,md,...,automatic,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...,California


In [4]:
data.isin(['Ring of Fire']).any()

alert           False
cdi             False
code            False
detail          False
dmin            False
felt            False
gap             False
ids             False
mag             False
magType         False
mmi             False
net             False
nst             False
place           False
rms             False
sig             False
sources         False
status          False
time            False
title           False
tsunami         False
type            False
types           False
tz              False
updated         False
url             False
parsed_place    False
dtype: bool

In [5]:
data['place'] = data['title'].str.extract(r', (.*$)')
data['place'].sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Australia', 'Azerbaijan', 'B.C., MX', 'Barbuda', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba ', 'British Virgin Islands',
       'Burma', 'CA', 'California', 'Canada', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Ecuador region',
       'El Salvador', 'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala',
       'Haiti', 'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Italy', 'Jamaica', 'Japan', 'Kansas',
       'Kentucky', 'Kyrgyzstan', 'Martinique', 'Mauritius', 'Mayotte',
       'Mexico', 'Missouri', 'Montana', 'NV', 'Nevada', 'New Caledonia',
       'New Hampshire', 'New Mexico', 'New Zealand', 'Nicaragua',
       'North Carolina', 'Northern Mariana Islands', 'Oklahoma', 'Oregon',
       'Pakistan', 'Papua New Guinea', 'Peru', 'Philippines',
       'Puerto Rico', 'Roman

1. Find the 95th percentile of earthquake magnitude in Japan using the
magType of 'mb'.

In [6]:
data.loc[data.magType == 'mb'].mag.quantile(0.95)

5.2

2. Find the percentage of earthquakes in Indonesia that were coupled with
tsunamis.

In [7]:
indonesia = data.loc[(data.place == 'Indonesia') & (data.type == 'earthquake'), ['type', 'tsunami', 'place']]
indonesia_tsunami = indonesia.loc[indonesia.tsunami != 0]
(indonesia_tsunami.tsunami.count()/indonesia.tsunami.count())*100

23.12925170068027

3. Get summary statistics for earthquakes in Nevada.

In [8]:
data.loc[(data.place.str.contains(r'NV|Nevada', na=False) & (data.type == 'earthquake'))].describe()

,cdi,dmin,felt,gap,mag,mmi,nst,rms,sig,time,tsunami,tz,updated
count,14.000000,647.000000,14.000000,647.000000,647.000000,1.00,647.000000,647.000000,647.000000,6.470000e+02,647.0,647.0,6.470000e+02
mean,2.421429,0.163155,2.500000,154.436615,0.437311,2.84,12.704791,0.140627,9.146832,1.538318e+12,0.0,-480.0,1.538409e+12
std,0.514675,0.161793,4.783787,69.474945,0.653397,NaN,10.052695,0.056765,17.939055,5.954980e+08,0.0,0.0,5.991682e+08
min,2.000000,0.001000,1.000000,29.140000,-0.500000,2.84,3.000000,0.000500,0.000000,1.537247e+12,0.0,-480.0,1.537323e+12
25%,2.000000,0.053000,1.000000,97.295000,-0.100000,2.84,6.000000,0.104400,0.000000,1.537859e+12,0.0,-480.0,1.537928e+12
50%,2.200000,0.109000,1.000000,150.040000,0.300000,2.84,9.000000,0.142900,1.000000,1.538286e+12,0.0,-480.0,1.538428e+12
75%,3.000000,0.223000,1.000000,200.515000,0.800000,2.84,16.000000,0.181050,10.000000,1.538824e+12,0.0,-480.0,1.538878e+12
max,3.300000,1.414000,19.000000,355.910000,2.900000,2.84,61.000000,0.340000,129.000000,1.539461e+12,0.0,-480.0,1.539483e+12


4. Add a column to the dataframe indicating whether or not the earthquake
happened in a country or US state that is on the Ring of Fire. Use:
Bolivia, Chile, Ecuador, Peru, Costa Rica, Guatemala, Mexico (be careful not to
select New Mexico), Japan, Philippines, Indonesia, New Zealand,
Antarctica (look for Antarctic), Canada, Fiji, Alaska, Washington,
California, Russia, Taiwan, Tonga, and Kermadec Islands.

In [15]:
data['Ring of Fire'] = (data['type'] == 'earthquake') & (data['place'].isin(
    ['Bolivia','Chile','Ecuador','Peru','Costa Rica','Guatemala','Mexico',
    'Japan','Philippines','Indonesia','New Zealand','Antarctic','Canada',
    'Fiji', 'Alaska','Washington','California','Russia','Taiwan','Tonga','Kermadec Islands']))
data[['type','title','place','Ring of Fire']].describe()

,type,title,place,Ring of Fire
count,9332,9332,9244,9332
unique,5,7807,92,2
top,earthquake,"M 0.4 - 10km NE of Aguanga, CA",Alaska,False
freq,9081,55,3658,5058


5. Calculate the number of earthquakes in the Ring of Fire locations and the
number outside them.

6. Find the tsunami count along the Ring of Fire.